In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


## Load existing dataset

In [2]:
# tract info
tract_info = pd.read_csv('DID_data/tract_info_complete_version.csv')

# matching info
matching_info = pd.read_csv('DID_data/matched_pairs_2021.csv')

In [3]:
tract_info.sample(2)

,Unnamed: 0,population,income,house_value,rent,tract,unemployment,poverty,tract_str,tract_str_last6,...,2021_med,2014_rec,2015_rec,2016_rec,2017_rec,2018_rec,2019_rec,2020_rec,2021_rec,total
837,837,4707,57439,482200,1742,7205,0.03,0.04,8041007205,7205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
438,438,3770,36394,340200,1379,1302,0.07,0.19,8031001302,1302,...,207.0,0.0,12.0,42.0,78.0,114.0,162.0,188.0,207.0,414.0


In [4]:
matching_info.sample(2)

,Unnamed: 0,population,income,house_value,rent,unemployment,poverty,dispensary_status,scores,match_id,record_id
551,920,5892,26613,171000,1067,0.03,0.21,0,0.568506,275,920
543,1353,4400,35198,211500,861,0.02,0.09,1,0.442306,271,1353


In [5]:
# census data 

year_info_df = []

for i in range(8):
    name = 'census_data/co_' + str(2014+i) +'.csv'
    data = pd.read_csv(name)
    data = data.drop('Unnamed: 0', axis=1)
    data['year'] = str(2014+i)
    year_info_df.append(data)
    

## Prepare new dataset

In [6]:
dispensaries = pd.merge(tract_info, matching_info,  how='inner', on=['population', 'income', 'house_value', 'rent'])
len(dispensaries)

726

In [7]:
dispensaries.isna().sum()

Unnamed: 0_x           0
population             0
income                 0
house_value            0
rent                   0
tract                  0
unemployment_x         0
poverty_x              0
tract_str              0
tract_str_last6        0
2014_med             370
2015_med             363
2016_med             359
2017_med             360
2018_med             346
2019_med             356
2020_med             352
2021_med             351
2014_rec             430
2015_rec             416
2016_rec             410
2017_rec             396
2018_rec             382
2019_rec             372
2020_rec             367
2021_rec             354
total                  0
Unnamed: 0_y           0
unemployment_y         0
poverty_y              0
dispensary_status      0
scores                 0
match_id               0
record_id              0
dtype: int64

In [8]:
dispensaries.fillna(0, inplace=True)

#### Reformat the dispensary data 

In [9]:
dispensaries_updated = pd.DataFrame(columns=['tract', 'year', 'rec', 'med', 'dispensary_status', 'match_id'])

for index, row in dispensaries.iterrows():
    for i in range(8):
        year = 2014+i
        rec = str(year) + '_rec'
        med = str(year) + '_med'
        dispensaries_updated.loc[len(dispensaries_updated)] = [str(int(row['tract_str'])), str(year), row[rec], row[med], row['dispensary_status'], row['match_id']]


In [10]:
len(dispensaries_updated)

5808

In [11]:
# export the data for future use
dispensaries_updated.to_csv('DID_data/tract_status.csv')

#### Concat all years' census data together

In [12]:
df_all_year = pd.concat(year_info_df)
df_all_year['tract'] = df_all_year['tract'].astype(str)

#### All tracts' census info, along with the cum store months every year

In [13]:
df_final = pd.merge(dispensaries_updated, df_all_year,  how='left', on=['tract', 'year'])
len(df_final)

5808

In [14]:
df_final.head(5)

,tract,year,rec,med,dispensary_status,match_id,population,income,house_value,rent,unemployment,poverty
0,8001007801,2014,0.0,0.0,0.0,180.0,4149.0,12675.0,156800.0,647.0,0.12,0.58
1,8001007801,2015,12.0,0.0,0.0,180.0,3930.0,12526.0,165800.0,665.0,0.07,0.56
2,8001007801,2016,24.0,0.0,0.0,180.0,4374.0,15885.0,162100.0,742.0,0.08,0.47
3,8001007801,2017,36.0,0.0,0.0,180.0,4412.0,17653.0,174300.0,818.0,0.10,0.44
4,8001007801,2018,48.0,0.0,0.0,180.0,4137.0,21320.0,208200.0,920.0,0.06,0.31


In [15]:
df_final.isna().sum()

tract                   0
year                    0
rec                     0
med                     0
dispensary_status       0
match_id                0
population           1110
income               1321
house_value          1116
rent                 1112
unemployment         1110
poverty              1110
dtype: int64

In [16]:
# fill the missing census data fields with previous values
df_final.fillna(method='ffill', inplace=True)

In [17]:
# export the data for future use
df_final.to_csv('DID_data/data_for_DID.csv')

#### Create a clean version of paired tracts 
    ['tract', 'dispensary_status', 'matched_tract']

In [18]:
"""
    Given a tract_id, return a list of all matched results
"""

def get_matched(tract_id, df):
    m_df = df[df['tract_str']==tract_id]
    match_ids = list(m_df.match_id)
    res = []
    for m_id in match_ids:
        valid_df = df[(df['match_id']==m_id) & (df['tract_str']!=tract_id)]
        res.append(valid_df.tract_str.values[0])
        
    return res

"""
    Find the next matched result
"""

def create_match_column(tract_id, match_count_dict, get_matched_tracts_dict):
    
    pos = match_count_dict[tract_id]
    res = get_matched_tracts_dict[tract_id][pos]
    match_count_dict[tract_id] = pos+1
    
    return res, match_count_dict





# prepare one dictionary for referencing the pair
# one dictionary for tracking the next matched record 
matched_pairs_df = dispensaries[['tract_str', 'dispensary_status', 'match_id']]
matched_pairs_df['tract_str'] = matched_pairs_df['tract_str'].astype(str)

get_matched_tracts_dict = {}
match_count_dict = {}
all_tracts = list(set(matched_pairs_df.tract_str))

for each_t in all_tracts:
    get_matched_tracts_dict[each_t] = get_matched(each_t, matched_pairs_df)
    match_count_dict[each_t] = 0
    

    
# create a cleaned version of each pair of matched tracts
matched_df_cleaned = pd.DataFrame(columns=['tract', 'dispensary_status', 'matched_tract'])

for index, row in matched_pairs_df.iterrows():
    curr_tract = row['tract_str']
    matched, match_count_dict = create_match_column(curr_tract, match_count_dict, get_matched_tracts_dict)
    matched_df_cleaned.loc[len(matched_df_cleaned)] = [str(curr_tract), row['dispensary_status'], matched]
 

In [19]:
matched_df_cleaned.head()

,tract,dispensary_status,matched_tract
0,8001007801,0,8041003801
1,8001007802,0,8041003002
2,8001007900,0,8041001800
3,8001007900,0,8083969301
4,8001008100,0,8031004506


In [20]:
# export the data for future use
matched_df_cleaned.to_csv('DID_data/matched_tracts_cleaned.csv')